# PROYECTO FINAL


Objetivos:
- Realizar la predicción de ORFs de la accesión NC_014976.1 del genoma completo de  'Bacillus subtilis'
- Comparar los resultados obtenidos con las herramientas orf_finder y pyrodigal para la predicción de ORFs
- Realizar la anotación del genoma con el servidor en línea prokka




Se empleó el módulo de python, "pyrodigal" v2.1.0 que interactúa con prodigal para correr la identificación de ORFs en Jupyter.
Se seleccionó el genoma de Bacillus subtilis NC_014976.1 de 4093599 nucleotidos.
Se empleó ORF_FINDER para comparar los resultados obtenidos con pyrodigal.
Se realizó la anotación con proka del genoma NC_014976.1

Se preparó el entorno de trabajo para pyrodigal:

In [14]:
import sys

!{sys.executable} -m pip install pyyaml biopython pyrodigal

Se cargan los paquetes a a utilizar

In [1]:
from pathlib import Path
from time import sleep
from typing import TextIO, Union
import pandas as pd
import pyrodigal
import requests
import yaml
from Bio import Entrez, SeqIO

Se empleó el archivo config.yaml para evitar compartir información personal. 
El contenido del archivo yaml es el siguiente:
email: "nina_lcl@gmail.com"
id: "NC_014976.1"
Y se colocó este archivo en la misma carpeta en donde se encuentra este notebook.

Si utilizamos el archivo yaml podemos acceder a los valores de la siguiente manera:

In [32]:
with open(Path.cwd() / "config.yaml") as f:
    config = yaml.safe_load(f)

In [33]:
Entrez.email = config["email"]

In [35]:
refseq_id = config["id"]

Se descargó el genoma completo en formato fasta 


In [36]:
def fetch_sequences(acc):
    try:
        handle = Entrez.efetch(db="nucleotide", id=acc, rettype="fasta")
        return handle
    except requests.HTTPError as err:
        print(err.response.status_code)
        print(err.response.text)
        sleep(60)


def get_fasta_sequences(acc_id):
    def save_fasta(x):
        handle = fetch_sequences(x)
        output_prefix = str(Path.cwd() / x)
        fasta_file = f"{output_prefix}.fasta"

        with open(fasta_file, "w") as f:
            f.write(handle.read())

    if isinstance(acc_id, list):
        [save_fasta(i) for i in acc_id]
    else:
        save_fasta(acc_id)

In [37]:
get_fasta_sequences(refseq_id)
#luego de este comando se descargó el archivo NC_014976.1.fasta en donde se encuentra el genoma completo

In [38]:
Bsub_prefix = str(Path.cwd() / refseq_id)
input_file = SeqIO.read(f"{Bsub_prefix}.fasta", "fasta")

In [39]:
orf_finder = pyrodigal.OrfFinder()
orf_finder.train(bytes(input_file.seq))

with open(f"{Bsub_prefix}.gff", "w") as dst:
    fasta_parser = SeqIO.parse(f"{Bsub_prefix}.fasta", "fasta")
    for i, record in enumerate(fasta_parser):
        genes = orf_finder.find_genes(bytes(record.seq))
        genes.write_gff(dst, sequence_id=record.id, header=(i == 0))

In [40]:
gff3_header = [
    "seqid",
    "source",
    "type",
    "start",
    "end",
    "score",
    "strand",
    "phase",
    "attributes",
]
#luego se genera el archivo NC_014976.1.gff en la carpeta donde se encuentra este notebook

In [11]:
df = pd.read_table(f"{Bsub_prefix}.gff", names=gff3_header, comment="#")

In [41]:
#Se pueden visualizar las primeros CDS de todo el listado generado, también indica la versión del pyrodigal empleado (pyrodigal v2.1.0)
df.head()

,seqid,source,type,start,end,score,strand,phase,attributes
0,NC_014976.1,pyrodigal_v2.1.0,CDS,3,1694,171.9,+,0,ID=NC_014976.1_1;partial=10;start_type=Edge;rb...
1,NC_014976.1,pyrodigal_v2.1.0,CDS,1798,2469,64.1,-,0,ID=NC_014976.1_2;partial=00;start_type=ATG;rbs...
2,NC_014976.1,pyrodigal_v2.1.0,CDS,2466,3173,80.2,-,0,ID=NC_014976.1_3;partial=00;start_type=ATG;rbs...
3,NC_014976.1,pyrodigal_v2.1.0,CDS,3160,4266,106.2,-,0,ID=NC_014976.1_4;partial=00;start_type=GTG;rbs...
4,NC_014976.1,pyrodigal_v2.1.0,CDS,4263,5636,107.6,-,0,ID=NC_014976.1_5;partial=00;start_type=GTG;rbs...


In [14]:
df.shape

(4054, 9)

Se identificaron 4054 CDS, los cuales también se pueden visualizar abriendo el archivo generado NC_014976.1.gff con el programa Notepad++

Se realizó la identificación de los ORFs para comparar los resultados obtenidos con pyrodigal

In [1]:
pip install credentials

Note: you may need to restart the kernel to use updated packages.


In [64]:
import credentials

In [14]:
#pip install biopython

In [65]:
from Bio import Entrez

In [ ]:
#pip install pandas

In [66]:
import pandas as pd

In [67]:
#orf_finder.ipynb
import pandas as pd
import numpy as np
import re
import credentials
import subprocess
from Bio import Entrez

In [68]:
#La siguiente estructura de datos te permite cambiar a otros códigos genéticos como por ejemplo de las mitocondrias que pueden usar el codón TGA para decodificar triptofano en vez de un codón de paro
genetic_code= {"AAA":"K","AAC":"N","AAG":"K","AAT":"N","ACA":"T","ACC":"T","ACG":"T","ACT":"T",
                "AGA":"R","AGC":"S","AGG":"R","AGT":"S","ATA":"I","ATC":"I","ATG":"M","ATT":"I",
                "CAA":"Q","CAC":"H","CAG":"Q","CAT":"H","CCA":"P","CCC":"P","CCG":"P","CCT":"P",
                "CGA":"R","CGC":"R","CGG":"R","CGT":"R","CTA":"L","CTC":"L","CTG":"L","CTT":"L",
                "GAA":"E","GAC":"D","GAG":"E","GAT":"D","GCA":"A","GCC":"A","GCG":"A","GCT":"A",
                "GGA":"G","GGC":"G","GGG":"G","GGT":"G","GTA":"V","GTC":"V","GTG":"V","GTT":"V",
                "TAA":"*","TAC":"Y","TAG":"*","TAT":"Y","TCA":"S","TCC":"S","TCG":"S","TCT":"S",
                "TGA":"*","TGC":"C","TGG":"W","TGT":"C","TTA":"L","TTC":"F","TTG":"L","TTT":"F"}

In [71]:
#Se colocó todos los nucleótidos en máyusculas. 
#Se reemplazó todas las bases por su reverso complementario, pero con minúsculas. Luego regresamos la secuencia a mayúsculas.
#Finalmente se pone la secuencia en reverso
def reverse_complement(sequence):
  sequence= sequence.upper().replace("A","t").replace("C","g").replace("G","c").replace("T","a").upper()[::-1]
  return sequence

In [73]:
#Se avanzó de 3 en 3 nucleótidos y se agregpó cada triplete a una lista de python
def get_codon_list(genome):
    codon_list = []
    for i in range(0, len(genome), 3):
        codon = genome[i:i+3]
        codon_list.append(codon)
    return codon_list

In [74]:
def get_nearest_start(stop_pos_df,start_pos_df,stop_pos):
    cur_stop_pos_index = stop_pos_df[stop_pos_df["stop_pos"]==stop_pos].index[0]
    if(cur_stop_pos_index==0):
        prev_stop_pos = 0
    elif(cur_stop_pos_index<len(stop_pos_df)):
        prev_stop_pos_index = cur_stop_pos_index - 1
        prev_stop_pos = stop_pos_df["stop_pos"][prev_stop_pos_index]
    cur_start_pos_index_list = start_pos_df[(start_pos_df["start_pos"]>=prev_stop_pos) & (start_pos_df["start_pos"]<stop_pos)]["start_pos"].index
    if (len(cur_start_pos_index_list)==0):
        cur_start_pos = np.nan
    else:
        cur_start_pos_index = cur_start_pos_index_list[0]
        cur_start_pos = start_pos_df["start_pos"][cur_start_pos_index]
    return cur_start_pos

In [75]:
def get_orf_pos_list(genome,frame,orf_size):
    genome_len = len(genome)
    if frame<0:
        genome = reverse_complement(genome)
        tmp_frame = -1 * frame
        genome = genome[tmp_frame-1:]
    else:
        genome = genome.upper()
        genome = genome[frame-1:]
    codon_list     = get_codon_list(genome)
    stop_pos_list  = []
    orf_pos_list   = []
    codon_df       = pd.DataFrame(codon_list,columns=["codon"])
    codon_df       = pd.DataFrame(codon_list,columns=["codon"])
    start_pos_list = list(codon_df[codon_df["codon"]=="ATG"].index)
    stop_pos_list  = list(codon_df[(codon_df["codon"]=="TAA") | (codon_df["codon"]=="TAG") | (codon_df["codon"]=="TGA")].index)
    start_pos_df   = pd.DataFrame(start_pos_list,columns=["start_pos"])
    stop_pos_df    = pd.DataFrame(stop_pos_list,columns=["stop_pos"])
    pos_df         = stop_pos_df.copy()
    pos_df["start_pos"] = pos_df["stop_pos"].apply(lambda x: get_nearest_start(stop_pos_df,start_pos_df,x))
    pos_df = pos_df[pos_df["start_pos"]>0]
    pos_df["orf_len"]   = pos_df["stop_pos"] - pos_df["start_pos"] + 1
    pos_df = pos_df[pos_df["orf_len"]>=orf_size]
    if frame >0:
        pos_df["orf_start_pos"] = ((pos_df["start_pos"] * 3) + 1) + (frame - 1)
        pos_df["orf_stop_pos"]  = ((pos_df["stop_pos"]  + 1) * 3) + (frame - 1)
    elif frame<0:
        pos_df["orf_start_pos"] = (genome_len + (1+frame) ) - ( pos_df["start_pos"]     * 3)
        pos_df["orf_stop_pos"]  = (genome_len + (1+frame) ) - ((pos_df["stop_pos"] + 1) * 3)
    orf_pos_list = pos_df[["orf_start_pos","orf_stop_pos"]].astype(int).values.tolist()
    return orf_pos_list

In [76]:
def get_orf_sequence(orf_df,genome,orf_id):
    strand    = list(orf_df[orf_df["id"]==orf_id]["strand"])[0]
    start_pos = int(orf_df[orf_df["id"]==orf_id]["start"])
    stop_pos  = int(orf_df[orf_df["id"]==orf_id]["stop"])    
    if(strand=="-"):
        orf_sequence = genome[start_pos:stop_pos]
        orf_sequence = reverse_complement(orf_sequence)
    else:
        orf_sequence = genome[start_pos-1:stop_pos]
    return orf_sequence

#- Se tomó la secuencia del ORF, que obtuvimos con `get_orf_sequence()`
#- Se utilizó `get_codon_list()` para convertir en una lista de codones
#- Se mapeó cada cada codón con el diccionario del código genético para determinar que aminoácido le corresponde a cada codón###


In [77]:
def translate_orf(orf_sequence):
    amino_acid_str = ""
    codon_list = get_codon_list(orf_sequence)
    for codon in codon_list:
        amino_acid = genetic_code[codon]
        amino_acid_str += amino_acid
    return amino_acid_str

#2. DESCARGA DE UN GENOMA
2.1. Se descargó un genoma usando la biblioteca de NCBI - Entrez (efech)

Se eliminó el encabezado de la secuencia y se dejó el genoma en una sola línea de texto

In [78]:
genome_name = "NC_014976.1"
#genome_name = "BSUB"
genome = Entrez.efetch(db="nucleotide", id=genome_name, rettype="fasta", retmode="text").read()
genome = re.sub(r"\>.*\n", "", genome).replace("\n","")

#2.2. Obtención de las posiciones de los ORFs mayores a "100" amino ácidos en cada uno de los seis marcos de lectura usando las funciones ya definidas en el paso 1

In [79]:
plus_one_pos_list    = get_orf_pos_list(genome, 1,100)
plus_two_pos_list    = get_orf_pos_list(genome, 2,100)
plus_three_pos_list  = get_orf_pos_list(genome, 3,100)
minus_one_pos_list   = get_orf_pos_list(genome,-1,100)
minus_two_pos_list   = get_orf_pos_list(genome,-2,100)
minus_three_pos_list = get_orf_pos_list(genome,-3,100)

#2.3. Construcción de dos dataframes, indicando los ORFs localizados en la cadena positiva del genoma y los ORFs localizados en la cadena negativa del genoma

In [80]:
plus_orf_pos_list  = plus_one_pos_list  + plus_two_pos_list  + plus_three_pos_list
minus_orf_pos_list = minus_one_pos_list + minus_two_pos_list + minus_three_pos_list
plus_orf_df = pd.DataFrame(plus_orf_pos_list,columns=["start","stop"])
minus_orf_df = pd.DataFrame(minus_orf_pos_list,columns=["stop","start"])

###2.4. Se agregó la información de la cadena a cada uno de los dataframes

#Con la última línea se concatenaron los ORFs luego de etiquetarlos 

In [81]:
plus_orf_df["strand"]  = "+"
minus_orf_df["strand"] = "-"
plus_orf_df = plus_orf_df[["start","stop","strand"]]
minus_orf_df = minus_orf_df[["start","stop","strand"]]
orf_df = pd.concat([plus_orf_df,minus_orf_df],ignore_index=True)

2.4.1. Al concatenar los dataframes, los ORFs no quedan ordenados de forma lógica, por lo que debemos ordenarlos con base en su coordenada de inicio

#Se les agregó una etiqueta a los ORF y se indicó su número de aparición en el genoma 

In [82]:
orf_df = orf_df.sort_values(by=["start"],ignore_index=True)
orf_df["genome"] = genome_name
orf_df["id"] = np.arange(len(orf_df)).astype(str)
orf_df["id"] = orf_df["genome"]+"."+orf_df["id"]

###2.5. Obtención de la secuencia codificante 

In [83]:
orf_df["orf_sequence"] = orf_df["id"].apply(lambda x: get_orf_sequence(orf_df,genome,x))

#2.6. Se empleó translate_orf() para agregar la traducción de la secuencia  

In [84]:
orf_df["orf_translation"] = orf_df["orf_sequence"].apply(lambda x: translate_orf(x))

Construcción de un archivo .gff

In [85]:
orf_df["info"] = "Id="+orf_df["id"]+";translation="+orf_df["orf_translation"]+";product=hypothetical protein"

2.7. Se agregó un par de columnas que indican que tipo de información y de que fuente se está teniendo dicha información

In [86]:
orf_df["score"]  = "."
orf_df["phase"]  = "."
orf_df["type"]   = "CDS"
orf_df["source"] = "python_orfinder"

Se empleó la función `.to_csv()` de pandas para almacenar la información del dataframe a un archivo tabular, 
en este punto ya tenemos la información necesaria para guardar nuestras predicciones en un archivo `.gff`**

In [87]:
orf_df[["genome","source","type","start","stop","score","strand","phase","info"]].to_csv(genome_name+".gff",sep="\t",header=False,index=False)

In [88]:
with open(genome_name+".faa","w") as file_name:
    fasta_str = ""
    for orf_id,orf_translation in list(orf_df[["id","orf_translation"]].values):
        fasta_str += ">" + orf_id + "\n" + orf_translation +"\n"
    file_name.write(fasta_str)

In [ ]:
!type NC_014976.1.gff
#este archivo también se puede abrir con el programa Notepad++

In [ ]:
! type NC_014976.1.faa

**PARTE 3

Se realizó la anotación del genoma NC_014976.1 empleando en servidor prokka para procariotas ubicado en https://usegalaxy.org/  y el archivo tipo fasta descargado inicialmente donde figura el genoma completo. Primero se carga el archivo utilizando la 
flecha de la izquierda "cargar datos" , haciendo clic en la opción "elegir archivos locales", seleccionando el tipo de archivo "fasta". Luego buscar prokka en la ventana de la izquierda y seleccionar el archivo cargado y la opción Bacteria.

Eggnog mapper  
También se evaluó el genoma en el servidor Eggnog mapper

http://eggnog-mapper.embl.de/submit_job
http://eggnog-mapper.embl.de/MM_vpez1efu/

# RESULTADOS



#PYRODIGAL:  4054 CDS pyrodigal v2.1.0
#ORF_FINDER: 4660 CDS python orf_finder
#EGGNOTE 511 CDS#
#Anotación PROKKA: De las 4157 secuencias detectadas (archivo Galaxy23-[Prokka_on_data_14__tsv].tabular)
            #4042 CDS prodigal:002006, 
            #83 AtRNA y 1 mtRNA con Aragorn, 
            #31 rRNA con barrnap:0.9



**REFERENCIAS:

Deng, Y., Zhu, Y., Wang, P., Zhu, L., Zheng, J., Li, R., ... & Sun, M. (2011). Complete genome sequence of Bacillus subtilis BSn5, an endophytic bacterium of Amorphophallus konjac with antimicrobial activity for the plant pathogen Erwinia carotovora subsp. carotovora.

Hyatt, D., Chen, GL., LoCascio, P.F. et al. Prodigal: prokaryotic gene recognition and translation initiation site identification. BMC Bioinformatics 11, 119 (2010). doi:10.1186/1471-2105-11-119.

Larralde, M., (2022). Pyrodigal: Python bindings and interface to Prodigal, an efficient method for gene prediction in prokaryotes. Journal of Open Source Software, 7(72), 4296. doi:10.21105/joss.04296.

Seemann, T. (2014). Prokka: rapid prokaryotic genome annotation. Bioinformatics, 30(14), 2068-2069. doi.org/10.1093/bioinformatics/btu153